In [0]:
import os 
import tarfile 
from six.moves import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Perform test train split using random sampling of data

Define the size of the test dataframe


In [0]:
SIZE=100

Generate the category labels for the dataset

In [0]:
# category 1 size is generated by adding a randomly generated integer between 0 and SIZE/6
cat1_size = int(SIZE/3) + np.random.randint(int(SIZE/6))

# category 2 size is generated by subtracting a randomly generated integer between 0 and SIZE/6
cat2_size = int(SIZE/3) - np.random.randint(int(SIZE/6))

# category 3 size is contrained by the SIZE
cat3_size = SIZE - cat1_size - cat2_size

# generate the category column by repeating the category labels '1', '2', and '3'
y=[1,]*cat1_size + [2,]*cat2_size + [3,]*cat3_size
# y

Generate the dataframe using indices as the main column and the category column "y"

In [36]:
dftest = pd.DataFrame({'test':np.arange(SIZE), 'cat':y  })
dftest[int(SIZE/3):int(SIZE/2)]

,test,cat
33,33,1
34,34,2
35,35,2
36,36,2
37,37,2
38,38,2
39,39,2
40,40,2
41,41,2
42,42,2


### Random split of data using np.random.permutation

Define splitratio and calculate the size of test sample

In [0]:
split_ratio=0.2
test_size=int(SIZE*split_ratio)

Shuffle the indices of the dataframe randomly

In [38]:
splt = np.random.permutation(SIZE)
splt

array([88, 72, 66, 37, 76, 45, 71, 78, 60, 74, 11, 19, 81, 67, 12, 98, 27,
       82, 54, 28, 48, 73, 49,  0, 94, 15, 83, 22,  1, 90, 16, 96, 57,  9,
       20, 18, 70, 53, 80,  6,  3, 26, 75, 29,  4, 58, 64, 34, 84, 99, 77,
       39, 92, 62, 42, 30, 35, 13, 32, 23, 69, 33, 63, 36, 43, 17, 25,  2,
       38, 52, 85, 61, 87,  5, 47, 59, 86, 68, 51, 24, 97, 31, 55, 40, 14,
       93,  8, 95, 41, 91, 50, 65, 44, 10, 46,  7, 79, 89, 56, 21])

Use shuffled indices to split data. The first few shuffled indices will select the test data randomly due to the random nature of those values 

In [0]:
train1, test1 = splt[test_size:], splt[:test_size]

In [40]:
dftest.iloc[test1]

,test,cat
88,88,3
72,72,3
66,66,3
37,37,2
76,76,3
45,45,2
71,71,3
78,78,3
60,60,3
74,74,3


In [41]:
dftest.iloc[train1].head() 

,test,cat
48,48,2
73,73,3
49,49,2
0,0,1
94,94,3


### Counting the distribution of the random samples

In [42]:
dftest.iloc[train1].groupby(['cat']).count()

,test
cat,
1,29
2,18
3,33


In [43]:
dftest.iloc[test1].groupby(['cat']).count()

,test
cat,
1,5
2,2
3,13


### Dataframe to compare the distribution of the random samples to the original distribution

In [44]:
dfratiotest = pd.DataFrame()
dfratiotest = dftest.groupby(['cat']).count()/SIZE
dfratiotest = dfratiotest.rename(columns={'test':'originalratio'})

dfratiotest['noShuffleSplit-test'] = dftest.iloc[test1].groupby(['cat']).count()/dftest.iloc[test1].groupby(['cat']).count().sum()
dfratiotest['noShuffleSplit-train'] = dftest.iloc[train1].groupby(['cat']).count()/dftest.iloc[train1].groupby(['cat']).count().sum()

dfratiotest

,originalratio,noShuffleSplit-test,noShuffleSplit-train
cat,,,
1,0.34,0.25,0.3625
2,0.20,0.10,0.2250
3,0.46,0.65,0.4125


## Looks like that random sampling does not preserve the relative ratios of the categories

# StratifiedShuffleSplit


Now we demonstrate how StratifiedShuffleSplit can split the data randomly into test-train sets while maintaining the relative ratio of the categories

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1,random_state=42,test_size=0.2)

In [46]:
# for loop is not necessary for splits with n_splits=1
for i,j in sss.split(np.zeros(SIZE),dftest['cat']):
    print(i,j)


[57 64 42 91 99  1 74 77 38 34 19 24 36 88 80 13  3 22 95 92 35 41 96 54
 26 30 11 63  2 16 89  9 37 70 98 68 97 47 18 82 94 27 17 72 76 59 66 25
 29 40 90 65 50 31 23 58 84 86 46  4 85 83 21 39 45  8 44 71 53 75  0 69
 12 15 32 87 93 56  5 52] [61 20 73 51 10 79 49 14 67 33 60  7 55 81 78 48 62  6 28 43]


The split returns only the indices. Passing the data as X has no effect on the output so we can use a placeholder like np.zeros() which reduces overhead for large datasets

In [0]:
train2, test2 = list(sss.split(np.zeros(SIZE),dftest['cat']))[0]

sss.split returns a generator so we have to convert that to a list by `list(sss.split())`. The list returned is a list of list of two arrays in the format  **[ [trainset1, testset1], [trainset2, testset2], [trainset3, testset3], ... ]**. To extract the first list set we use `list(sss.split())[0]`

### Counting the distribution of the random samples

In [48]:
dftest.iloc[train2].groupby(['cat']).count()

,test
cat,
1,27
2,16
3,37


In [49]:
dftest.iloc[test2].groupby(['cat']).count()

,test
cat,
1,7
2,4
3,9


### Dataframe to compare the distribution of the random samples to the original distribution

In [50]:
dfratiotest['StratifiedShuffleSplit-test'] = dftest.iloc[test2].groupby(['cat']).count()/dftest.iloc[test2].groupby(['cat']).count().sum()
dfratiotest['StratifiedShuffleSplit-train'] = dftest.iloc[train2].groupby(['cat']).count()/dftest.iloc[train2].groupby(['cat']).count().sum()
dfratiotest

,originalratio,noShuffleSplit-test,noShuffleSplit-train,StratifiedShuffleSplit-test,StratifiedShuffleSplit-train
cat,,,,,
1,0.34,0.25,0.3625,0.35,0.3375
2,0.20,0.10,0.2250,0.20,0.2000
3,0.46,0.65,0.4125,0.45,0.4625


Stratified sampling came close to the original distribution but not quite. We now calculate how far off is each sampling method from the original distribution of data among the categories. We define the error rate as the relative deviation from the original values i.e. $ \epsilon = \Delta c_i / c_i $

In [51]:
dferror = pd.DataFrame()
for col in dfratiotest.columns[1:]:
    dferror[col] = (dfratiotest[col]-dfratiotest['originalratio'])/dfratiotest['originalratio']*100

dferror

,noShuffleSplit-test,noShuffleSplit-train,StratifiedShuffleSplit-test,StratifiedShuffleSplit-train
cat,,,,
1,-26.470588,6.617647,2.941176,-0.735294
2,-50.000000,12.500000,0.000000,0.000000
3,41.304348,-10.326087,-2.173913,0.543478


We see that the error rate is considerably lower for StratifiedShuffleSplit. For smaller datasets StratifiedShuffleSplit performs monly slightly better than random sampling. However, as the datasets grow in size, StratifiedShuffleSplit performs considerably better than just random sampling. Try running the whole notebook by having different values of hte parameter SIZE and see the difference.
